In [23]:
import numpy as np
import lzma
import multiprocessing
num_proc = multiprocessing.cpu_count()

def get_compression_rate(example):
  tokens = np.load(example['path']).astype(np.int16)
  compressed = lzma.compress(tokens.T.ravel().tobytes()) # transposing increases compression ratio ;)
  example.update({'compression_rate': (tokens.size * 10 / 8) / len(compressed)}) # 10 bits per token
  return example
print("compression ratio for example: ", get_compression_rate({'path': "../examples/tokens.npy"})['compression_rate']) # 10 bits per token

compression ratio for example:  1.4792899408284024


In [24]:
# load split 0,1,2,3 of the commavq dataset
from datasets import load_dataset, DatasetDict
splits = ['0', '1', '2', '3']
ds = load_dataset('commaai/commavq', num_proc=num_proc, split=splits)
ds = DatasetDict(zip(splits, ds))

In [25]:
ratios = ds.map(get_compression_rate, desc="computing compression ratio", num_proc=num_proc, load_from_cache_file=False)

computing compression ratio (num_proc=64):   0%|          | 0/2500 [00:00<?, ? examples/s]

computing compression ratio (num_proc=64):   0%|          | 0/2500 [00:00<?, ? examples/s]

computing compression ratio (num_proc=64):   0%|          | 0/2500 [00:00<?, ? examples/s]

computing compression ratio (num_proc=64):   0%|          | 0/2500 [00:00<?, ? examples/s]

In [26]:
print("compression ratio: ", np.mean([ratios[s]['compression_rate'] for s in splits]))

compression ratio:  1.6310833787228878
